In [71]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import pytz
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

try:
  # Use the %tensorflow_version magic if in colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [72]:
X_train = pd.read_csv('../datasets/scaled_train_test/X_train_standard.csv', delimiter=',')
X_test = pd.read_csv('../datasets/scaled_train_test/X_test_standard.csv', delimiter=',')
y_train = X_train['Value (kWh)']
y_test = X_test['Value (kWh)']
X_train.drop(['Value (kWh)'], axis=1, inplace=True)
X_test.drop(['Value (kWh)'], axis=1, inplace=True)

In [73]:
X_test.columns

Index(['Pressure (msl) (hPa)', 'Relative humidity (%)',
       'Air temperature (degC)', 'Dew-point temperature (degC)',
       'Wind speed (m/s)', 'Global radiation (W/m2)', 'Wind_direction',
       'Cloud_amount_binary', 'Precipitation_binary', 'hour_of_day',
       'day_of_year', 'solar_elev_angle', 'sun_azimuth'],
      dtype='object')

In [74]:
import tensorflow as tf
keras = tf.keras
keras.backend.clear_session()
# tf.random.set_seed(42)
np.random.seed(42)

In [75]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

We will split the X_train data further into X_train and X_val, which is going to be our validation set.

In [76]:
from sklearn.model_selection import train_test_split

In [77]:
X_train_v, X_val, y_train_v, y_val = train_test_split(X_train, y_train, random_state=42, test_size=0.8) 

In [81]:
nn = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu', input_shape=X_train_v.shape[1:]),
    keras.layers.Dense(1)
])
nn.compile(loss='mean_squared_error', optimizer='sgd')
history= nn.fit(X_train_v, y_train_v, epochs=20, validation_data=(X_val,y_val))

ValueError: could not broadcast input array from shape (13,1) into shape (13)

In [64]:
nn = keras.models.Sequential()
nn.add(keras.layers.Dense(30, activation='relu', input_shape=X_train_v.shape[1:]))
nn.add(keras.layers.Dense(1))
lr_schedule = keras.callbacks.LearningRateScheduler(lambda epoch: 1e-8 * 10**(epoch / 20))
reset_states = ResetStatesCallback()
optimizer = keras.optimizers.SGD(lr=1e-8, momentum=0.9)
nn.compile(optimizer=optimizer, loss='mean_squared_error')

In [65]:
X_train.shape, X_train_v.shape, X_val.shape

((33796, 13), (6759, 13), (27037, 13))

In [66]:
X_train.shape

(33796, 13)

In [67]:
X_train.ndim

2

ValueError: cannot reshape array of size 87867 into shape (13,)

In [84]:
y_train_v.shape

(6759,)

In [82]:
history = nn.fit(X_train_v,y_train_v, epochs=100,
                 validation_data=(X_val,y_val),
                callbacks=[lr_schedule, reset_states])

ValueError: could not broadcast input array from shape (13,1) into shape (13)

Read the proper learning rate, typically at the elbow of the graph.

In [ ]:
plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-8, 1e-3, 0,10])

In [ ]:
Selected learning rate: __FILL__

#### Training NN model
The model will be training for maximum 500 epochs, saving itself each time it improves in __'my_checkpoint.h5'__. If there is no more improvement for the next 50 epochs, the __EarlyStopping__ callback will finish the training and use the checkpointed best model. 

In [ ]:
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.LSTM(64,
                                    return_sequences=True,
                                    input_shape=x_train_multi.shape[-2:]))
nn_model.add(tf.keras.layers.LSTM(32, return_sequences=True, activation='relu'))
nn_model.add(tf.keras.layers.LSTM(16, return_sequences=False, activation='relu'))
nn_model.add(tf.keras.layers.Dense(future_target))
nn_model.add(tf.keras.layers.Lambda(lambda x: x * 200.0))
optimizer = keras.optimizers.SGD(lr=0.5e-5, momentum=0.9)
nn_model.compile(optimizer=optimizer, loss=keras.losses.Huber(), metrics=['mae'])
reset_states = ResetStatesCallback()
model_checkpoint = keras.callbacks.ModelCheckpoint(
    "my_checkpoint.h5", save_best_only=True)
early_stopping = keras.callbacks.EarlyStopping(patience=50)
nn_model.fit(train_data_multi, epochs=500, steps_per_epoch=200,
                                          validation_data=val_data_multi,
                                          validation_steps=50, callbacks=[early_stopping, model_checkpoint, reset_states])


In [ ]:
model = keras.models.load_model("my_checkpoint.h5")